<a href="https://colab.research.google.com/github/zyntonyson/Erisk2017/blob/master/00_Depression_data_process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Verificar que haya concordancia con los archivos xml originales




In [0]:
import os
path_data='/content/drive/My Drive/MCE/Participaciones/05.Estancia Jun19/Practicas NLP/Deteccion de depresion/Depression/reddit2017'
#path_data=path_data.replace("\\" , "/" )+'/'
#path_data

#Obtener listado del primer chunk y verificar que se encuentre en el resto
   
individuos_train_p=list(os.listdir(path_data+'/training/positive/chunk_1/'))
individuos_train_p= [individuo.replace('1.xml','') for individuo in individuos_train_p if individuo !='desktop.ini']
print(len(individuos_train_p))
print('\n Respuestas positvas entrenamiento')   
for chunk in range(1,11):
    individ_in_chunk_p=os.listdir(path_data+'/training/positive/chunk_'+str(chunk))
    individ_in_chunk_p=[individuo.replace(str(chunk)+'.xml','') for individuo in individ_in_chunk_p if individuo !='desktop.ini']
    print('chunk:',chunk,'\t',set(individuos_train_p)==set(individ_in_chunk_p))
    #print('\t',len(individ_in_chunk_p))


individuos_train_n=list(os.listdir(path_data+'/training/negative/chunk_1/'))
individuos_train_n= [individuo.replace('1.xml','') for individuo in individuos_train_n if individuo !='desktop.ini']
print(len(individuos_train_n))
print('\n Respuestas negativas entrenamiento')
for chunk in range(1,11):
    individ_in_chunk_n=os.listdir(path_data+'/training/negative/chunk_'+str(chunk))
    individ_in_chunk_n=[individuo.replace(str(chunk)+'.xml','') for individuo in individ_in_chunk_n if individuo !='desktop.ini']
    print('chunk:',chunk,'\t',set(individuos_train_n)==set(individ_in_chunk_n))
    #print(individ_in_chunk_n)
    #print(individuos_train_n)


individuos_train_t=list(os.listdir(path_data+'/testing/chunk_1/'))
individuos_train_t= [individuo.replace('1.xml','') for individuo in individuos_train_t if individuo !='desktop.ini']
print(len(individuos_train_t))
print('\n Respuestas test entrenamiento')
for chunk in range(1,11):
    individ_in_chunk_t=os.listdir(path_data+'/testing/chunk_'+str(chunk))
    individ_in_chunk_t=[individuo.replace(str(chunk)+'.xml','') for individuo in individ_in_chunk_t if individuo !='desktop.ini']
    print('chunk:',chunk,'\t',set(individuos_train_t)==set(individ_in_chunk_t))


83

 Respuestas positvas entrenamiento
chunk: 1 	 True
chunk: 2 	 True
chunk: 3 	 True
chunk: 4 	 True
chunk: 5 	 True
chunk: 6 	 True
chunk: 7 	 True
chunk: 8 	 True
chunk: 9 	 True
chunk: 10 	 True
403

 Respuestas negativas entrenamiento
chunk: 1 	 True
chunk: 2 	 True
chunk: 3 	 True
chunk: 4 	 True
chunk: 5 	 True
chunk: 6 	 True
chunk: 7 	 True
chunk: 8 	 True
chunk: 9 	 True
chunk: 10 	 True
401

 Respuestas test entrenamiento
chunk: 1 	 True
chunk: 2 	 True
chunk: 3 	 True
chunk: 4 	 True
chunk: 5 	 True
chunk: 6 	 True
chunk: 7 	 True
chunk: 8 	 True
chunk: 9 	 True
chunk: 10 	 True


## Procesamiento del corpus

### Características del corpus:

1. Post de usuarios de reddit
1. Separados en 10 chunks
1. Formato XML

#### Datos de entrenamiento



In [0]:
import re
import os
mypath='/content/drive/My Drive/MCE/Participaciones/05.Estancia Jun19/Practicas NLP/Deteccion de depresion/Actualizacion sep 2019'
os.chdir( mypath )
from os import listdir
from glob import glob # Listar archivos

# Scrapping
from bs4 import BeautifulSoup as Soup
import requests
import pandas as pd 
import numpy as np


In [0]:
#Preproceso
import nltk
# Convertir a palabras
from nltk.tokenize import word_tokenize
# Listado de stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
#Steeming
from nltk.stem import PorterStemmer
ps=PorterStemmer()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
def corpus_process_train(nChunks,stemm=True,stopword=True,maxlenword=15):
    
    #Directorios Raiz
    root_p='/content/drive/My Drive/MCE/Participaciones/05.Estancia Jun19/Practicas NLP/Deteccion de depresion/Depression/reddit2017/training/positive/chunk_'
    root_n='/content/drive/My Drive/MCE/Participaciones/05.Estancia Jun19/Practicas NLP/Deteccion de depresion/Depression/reddit2017/training/negative/chunk_'
    
    #Lista de sujetos por tipo de respuesta
    subjects_p=[subject.replace('_1.xml','') for subject in listdir(root_p+'1/') if subject !='desktop.ini']
    subjects_n=[subject.replace('_1.xml','') for subject in listdir(root_n+'1/') if subject !='desktop.ini']
    
    #DataFrame contenedor
    # Se crea en un principio unicamente con los ID's de los sujetos y su etiqueta
    df_train=pd.DataFrame(subjects_p+subjects_n,columns=['Id']) # Id_sujetos
    df_train['Depress']=len(subjects_p)*[1]+ len(subjects_n)*[0] #Etiqueta depression
    
    # Procesar cada chunk para generar dataglobal
    
    for chunk in range(1,nChunks+1):
        subjects_p_path=[root_p+str(chunk)+'/'+ subject+'_'+str(chunk)+'.xml' for subject in subjects_p]
        subjects_n_path=[root_n+str(chunk)+'/'+ subject+'_'+str(chunk)+'.xml' for subject in subjects_n]
        subjects_train_path= subjects_p_path +subjects_n_path
        chunk_posts=[]
        for subject in subjects_train_path:
            posts_subject=[]
            infile=open(subject,'r',encoding='utf8')
            post_raw=Soup(infile.read())
            posts=post_raw.find_all('text')               #Obtener todos los post
            for post in posts:
                if post.getText()!=None:
                    posts_subject.append(post.getText())
                    
#########################################Preproceso############################################################
            # 1.Minusculas  2. Stemming 3. Solo palabras        
            all_post_subject=' '.join(posts_subject)
            #all_post_subject=re.sub(r'http\S+', '', all_post_subject)  #Eliminar direcciones web is.alpha() realiza la tarea
            tokens_all_post_subject=word_tokenize(all_post_subject)    #Tokenizar
            tokens_subject=[token.lower()  for token in tokens_all_post_subject]
            if stemm:
              if stopword:
                tokens_subject=[ps.stem(token) for token in tokens_subject if token.isalpha() and not token in stop_words and len(token)<maxlenword]
              else:
                tokens_subject=[ps.stem(token) for token in tokens_subject if token.isalpha() and len(token)<maxlenword]
            else:
              if stopword:
                tokens_subject=[token for token in tokens_subject if token.isalpha() and not token in stop_words and len(token)<maxlenword]
              else:
                tokens_subject=[token for token in tokens_subject if token.isalpha() and len(token)<maxlenword]
            post_clean=' '.join(tokens_subject)  # Unir todos los post
            chunk_posts.append(post_clean)
        name_chunk='Chunk_'+str(chunk)
        df_train[name_chunk]=chunk_posts
   
    return df_train

#### Datos de prueba

In [0]:
def corpus_process_test(nChunks,stemm=True,stopword=True,maxlenword=15):
    #Procesar labels
    dir_data_test='/content/drive/My Drive/MCE/Participaciones/05.Estancia Jun19/Practicas NLP/Deteccion de depresion/Depression/reddit2017/test_golden_truth.txt'
    y_test = pd.read_csv(dir_data_test, header = None, sep= ' ')
    y_test=y_test.drop([0], axis=1)
    y_test.columns=['Id','Depress']

    #Directorio Raiz
    root_p='/content/drive/My Drive/MCE/Participaciones/05.Estancia Jun19/Practicas NLP/Deteccion de depresion/Depression/reddit2017/testing/chunk_'
    #Lista de sujetos por tipo de respuesta
    subjects_t=[subject.replace('_1.xml','') for subject in listdir(root_p+'1/') if subject !='desktop.ini']
    
    #DataFrame contenedor
    df_train=pd.DataFrame(subjects_t,columns=['Id']) # Id_sujetos
    
    # Procesar cada chunk para generar dataglobal
    
    for chunk in range(1,nChunks+1):
        subjects_t_path=[root_p+str(chunk)+'/'+ subject+'_'+str(chunk)+'.xml' for subject in subjects_t]
        chunk_posts=[]
        for subject in subjects_t_path:
            posts_subject=[]
            infile=open(subject,'r',encoding='utf8')
            post_raw=Soup(infile.read())
            posts=post_raw.find_all('writing')               #Obtener todos los post
            for post in posts:
              if(post.find('title').getText()!=None):
                posts_subject.append(post.find('title').getText())
              if(post.find('text').getText()!=None):
                posts_subject.append(post.find('text').getText())                    
#########################################Preproceso############################################################
            # 1.Minusculas  2. Stemming 3. Solo palabras        
            all_post_subject=' '.join(posts_subject)
            #all_post_subject=re.sub(r'http\S+', '', all_post_subject)  #Eliminar direcciones web, .isalpha() realiza la tarea
            tokens_all_post_subject=word_tokenize(all_post_subject)    #Tokenizar
            tokens_subject=[token.lower()  for token in tokens_all_post_subject]
            if stemm:
              if stopword:
                tokens_subject=[ps.stem(token) for token in tokens_subject if token.isalpha() and not token in stop_words and len(token)<maxlenword]
              else:
                tokens_subject=[ps.stem(token) for token in tokens_subject if token.isalpha() and len(token)<maxlenword]
            else:
              if stopword:
                tokens_subject=[token for token in tokens_subject if token.isalpha() and not token in stop_words and len(token)<maxlenword]
              else:
                tokens_subject=[token for token in tokens_subject if token.isalpha() and len(token)<maxlenword]
            post_clean=' '.join(tokens_subject)  # Unir todos los post
            chunk_posts.append(post_clean)
        name_chunk='Chunk_'+str(chunk)
        df_train[name_chunk]=chunk_posts
    df_test=pd.merge(y_test,df_train,on='Id',how='inner')
    return df_test


In [0]:
df_train=corpus_process_train(10,stemm=False)

In [0]:
df_test=corpus_process_test(10,stemm=False)

In [0]:
df_test.isnull().sum()

Id          0
Depress     0
Chunk_1     0
Chunk_2     0
Chunk_3     0
Chunk_4     0
Chunk_5     0
Chunk_6     0
Chunk_7     0
Chunk_8     0
Chunk_9     0
Chunk_10    0
dtype: int64

In [0]:
# Salvar datos
df_train.to_csv('train_Depression_all_chunks_nosteem.csv',encoding='utf-8',na_rep= ' ')
df_test.to_csv('test_Depression_all_chunks_nosteem.csv')

In [0]:
train_df=pd.read_csv('train_Depression_all_chunks_nosteem.csv')
train_df=train_df.replace(np.nan, '', regex=True)
test_df=pd.read_csv('test_Depression_all_chunks_nosteem.csv')
test_df=train_df.replace(np.nan, '', regex=True)

In [0]:

train_df.isnull().sum()

Unnamed: 0    0
Id            0
Depress       0
Chunk_1       0
Chunk_2       0
Chunk_3       0
Chunk_4       0
Chunk_5       0
Chunk_6       0
Chunk_7       0
Chunk_8       0
Chunk_9       0
Chunk_10      0
dtype: int64

In [0]:
test_df.isnull().sum()

Unnamed: 0    0
Id            0
Depress       0
Chunk_1       0
Chunk_2       0
Chunk_3       0
Chunk_4       0
Chunk_5       0
Chunk_6       0
Chunk_7       0
Chunk_8       0
Chunk_9       0
Chunk_10      0
dtype: int64